# NLP (comentários página do Facebook)
Este notebook contém o código referente ao modelo que analisa sentimentos em formato textual. A proposta aqui foi a de criar um modelo que pudesse analisar os comentários públicos da página do Facebook da Passos Mágicos e assim demonstrar o quão impactante suas ações são nas vidas das pessoas. Foram considerados 3 categorias de comentários: positivo, negativo e neutro.

Aqui vale notar que além dos comentários da página do Facebook da ONG, também foram gerados batches de comentários pelo ChatGPT da OpenAI. Isso se fez necessário, pois não haviam muitos comentários ou reviews da ONG que pudessem ser utilizados para treinar efetivamente um NLP.

Os comentários e reviews originais da página da ONG foram misturados juntos dos comentários gerados pelo ChatGPT.

In [147]:
# para tratar datasets desbalanceados
!pip install imbalanced-learn

In [148]:
# para baixar datasets diretamente do Kaggle
!pip install kaggle

In [149]:
# para utilizar no lemmatizer com um dicionário em pt-BR
!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 49.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [150]:
import pandas as pd
import numpy as np
import nltk
import re
import string
import unicodedata
import spacy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from imblearn.under_sampling import RandomUnderSampler
import joblib
import os
from google.colab import userdata

In [151]:
# vars de ambiente para o Kaggle (aqui também seria possível utilizar o Drive do Google como repositório para a chave)
# o único ponto é que utilizando vars de ambiente, preciso obrigatoriamente utilizar o pacote 'os'
os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')
os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')

pd.set_option('display.max_columns', None)

In [152]:
# Kaggle pt-BR dataset
os.system('mkdir /content/kaggle-data')
os.system('kaggle datasets download -d augustop/portuguese-tweets-for-sentiment-analysis -p /content/kaggle-data')
os.system('unzip /content/kaggle-data/portuguese-tweets-for-sentiment-analysis.zip -d /content/kaggle-data')

256

In [153]:
# NTLK
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [154]:
print(spacy.util.get_installed_models())

['pt_core_news_sm', 'en_core_web_sm']


In [155]:
nlp_spacy_ptbr = spacy.load("pt_core_news_sm")

## Funções utilitárias para limpeza e normalização de strings

In [156]:
def normalize_accents(text):
    return unicodedata.normalize("NFKD", text).encode("ASCII", "ignore").decode("utf-8")

def remove_punctuation(text):
    punctuations = string.punctuation
    table = str.maketrans({key: " " for key in punctuations})
    text = text.translate(table)
    return text

def normalize_str(text):
    text = text.lower()
    text = remove_punctuation(text)
    text = normalize_accents(text)
    text = re.sub(re.compile(r" +"), " ",text)
    return " ".join([w for w in text.split()])

def tokenizer(text):
    stop_words = stopwords.words("portuguese")
    if isinstance(text, str):
        text = normalize_str(text)
        text = "".join([w for w in text if not w.isdigit()])
        text = word_tokenize(text)
        text = [x for x in text if x not in stop_words]
        text = [y for y in text if len(y) > 2]
        return " ".join([t for t in text])
    else:
        return None

def lemmatize(text):
    doc = nlp_spacy_ptbr(text)
    return " ".join([token.lemma_ for token in doc])

## Carregando os comentários e reviews da página do Facebook da ONG Passos Mágicos + comentários gerados pelo ChatGPT da OpenAI

In [157]:
df_1 = pd.read_csv('/content/chatgpt-base-comentarios-negativos.csv', sep=';')
df_2 = pd.read_csv('/content/chatgpt-base-comentarios-neutros.csv', sep=';')
df_3 = pd.read_csv('/content/chatgpt-base-comentarios-positivos.csv', sep=';')
df_4 = pd.read_csv('/content/facebook-mentions.csv', sep=';')
df_5 = pd.read_csv('/content/facebook-reviews.csv', sep=';')
df_ptbr_sentiment_1 = pd.read_csv('/content/kaggle-data/TweetsNeutralHash.csv')
df_ptbr_sentiment_2 = pd.read_csv('/content/kaggle-data/TweetsNeutralNews.csv')
df_ptbr_sentiment_3 = pd.read_csv('/content/kaggle-data/NoThemeTweets.csv')

In [158]:
df_ptbr_sentiment_1['categoria'] = df_ptbr_sentiment_1['sentiment'].str.lower()
df_ptbr_sentiment_1['texto'] = df_ptbr_sentiment_1['tweet_text']
df_ptbr_sentiment_1_final = df_ptbr_sentiment_1[['texto', 'categoria']]
df_ptbr_sentiment_1_final = df_ptbr_sentiment_1_final.sample(15000, random_state=777)

In [159]:
df_ptbr_sentiment_2['categoria'] = df_ptbr_sentiment_2['sentiment'].str.lower()
df_ptbr_sentiment_2['texto'] = df_ptbr_sentiment_2['tweet_text']
df_ptbr_sentiment_2_final = df_ptbr_sentiment_2[['texto', 'categoria']]
df_ptbr_sentiment_2_final = df_ptbr_sentiment_2_final.sample(15000, random_state=777)

In [160]:
df_ptbr_sentiment_3['categoria'] = df_ptbr_sentiment_3['sentiment'].str.lower()
df_ptbr_sentiment_3['texto'] = df_ptbr_sentiment_3['tweet_text']
df_ptbr_sentiment_3_final = df_ptbr_sentiment_3[['texto', 'categoria']]
df_ptbr_sentiment_3_final = df_ptbr_sentiment_3_final.sample(60000, random_state=777)

In [161]:
df = pd.concat([df_1, df_2, df_3, df_4, df_5, df_ptbr_sentiment_1_final, df_ptbr_sentiment_2_final, df_ptbr_sentiment_3_final], ignore_index=True)
df

,texto,categoria
0,O atendimento foi péssimo.,negativo
1,Produto de baixa qualidade.,negativo
2,Muito insatisfeito com a compra.,negativo
3,Demorou muito para chegar.,negativo
4,Chegou quebrado e não funciona.,negativo
...,...,...
91055,Nossa :(( https://t.co/QTBjmJ6TGx,negativo
91056,"@vit_bm uai, acho que foi uma gastroenterite :(",negativo
91057,@powerPeraltan yay :D vou adorar fazer essa co...,positivo
91058,@lariazvd021 Ohhh obrigado larissa!! Com certe...,positivo


In [162]:
df['categoria'].unique()

array(['negativo', 'neutro', 'positivo'], dtype=object)

In [163]:
df['categoria'].value_counts()

categoria
negativo    40026
neutro      30443
positivo    20591
Name: count, dtype: int64

In [164]:
df.shape

(91060, 2)

In [165]:
df.head()

,texto,categoria
0,O atendimento foi péssimo.,negativo
1,Produto de baixa qualidade.,negativo
2,Muito insatisfeito com a compra.,negativo
3,Demorou muito para chegar.,negativo
4,Chegou quebrado e não funciona.,negativo


## Undersample da amostra para balancear as classes de treinamento

In [166]:
X = df.drop('categoria', axis=1)
y = df['categoria']

In [167]:
rus = RandomUnderSampler(random_state=777)
X_rus, y_rus = rus.fit_resample(X, y)
df_rus = pd.concat([X_rus, y_rus], axis=1)

df_rus['categoria'].value_counts()

categoria
negativo    20591
neutro      20591
positivo    20591
Name: count, dtype: int64

## Randomização, tokenização e lematização do dataset

In [168]:
%%time
df_rus['texto_processado'] = df_rus['texto'].apply(tokenizer)
df_rus['texto_processado_lemm'] = df_rus['texto_processado'].apply(lemmatize)
df_rus.head()

CPU times: user 10min 51s, sys: 5.08 s, total: 10min 56s
Wall time: 11min 20s


,texto,categoria,texto_processado,texto_processado_lemm
56918,EU QUERO GRITAR PRA TODO MUNDO SABER QUE EU TE...,negativo,quero gritar pra todo mundo saber amo tudo voc...,querer gritar pra todo mundo saber amo tudo vo...
64228,Eu tenho cara vez mais nojo do ser humano :( h...,negativo,cara vez nojo humano https xqnjtmr,Cara vez nojo humano https xqnjtmr
80818,@kommurolla Fala :((,negativo,kommurolla fala,kommurollo falar
64400,@FuuDigo queria tanto :( mas tenho q economiza...,negativo,fuudigo queria tanto economizar porque quero f...,Fuudigo querer tanto economizar porque querer ...
88493,@trxart não eu esqueci :(,negativo,trxart nao esqueci,trxart nao esqueci


In [169]:
df_final = shuffle(df_rus)
df_final.reset_index(drop=True, inplace=True)

In [170]:
df_final.head()

,texto,categoria,texto_processado,texto_processado_lemm
0,@soo_iie Acabo de comprar 30 :),positivo,soo iie acabo comprar,soo iie acabo comprar
1,"Agora é só marcar na agenda, reservar o copo d...",positivo,agora marcar agenda reservar copo chope cair f...,agora marcar agenda reservar copo chope cair f...
2,@Thaisandd @Pornfood18 Quero meu strogonoff pr...,positivo,thaisandd pornfood quero strogonoff primeiro,thaisandd Pornfood querer strogonoff primeiro
3,#NOVIDADE #TheMeg Confiram os comentarios sobr...,neutro,novidade themeg confiram comentarios sobre fil...,novidade themeg confir comentario sobre filme ...
4,Petrobras manterá preço médio da gasolina na s...,neutro,petrobras mantera preco medio gasolina sexta f...,petrobras mantera preco medio gasolinar sexto ...


## Dividindo o dataset em treino e teste

In [171]:
X = df_final['texto_processado_lemm'].values
y = df_final['categoria'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=777, stratify=y)

In [172]:
print('X_train:', X_train.size)
print('X_test:', X_test.size)
print('y_train:', y_train.size)
print('y_test:', y_test.size)

X_train: 49418
X_test: 12355
y_train: 49418
y_test: 12355


In [173]:
%%time
vect = CountVectorizer(lowercase=False)
vect.fit(X_train)

CPU times: user 937 ms, sys: 9.74 ms, total: 947 ms
Wall time: 958 ms


CountVectorizer(lowercase=False)

In [174]:
%%time
X_train = vect.transform(X_train)
X_train

CPU times: user 734 ms, sys: 5 ms, total: 739 ms
Wall time: 754 ms


<49418x84963 sparse matrix of type '<class 'numpy.int64'>'
	with 446643 stored elements in Compressed Sparse Row format>

In [175]:
%%time
X_test = vect.transform(X_test)
X_test

CPU times: user 176 ms, sys: 1.87 ms, total: 178 ms
Wall time: 182 ms


<12355x84963 sparse matrix of type '<class 'numpy.int64'>'
	with 95206 stored elements in Compressed Sparse Row format>

## Utilização do modelo SVM para categorizar as frases

In [176]:
%%time
modelo = SVC(kernel='linear')
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)

CPU times: user 24min 10s, sys: 4.16 s, total: 24min 14s
Wall time: 24min 38s


In [177]:
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.7889923108053419


In [178]:
y_pred_class = [x for x in y_pred]
y_test_class = y_test
print(classification_report(y_test_class, y_pred_class))

              precision    recall  f1-score   support

    negativo       0.70      0.76      0.73      4118
      neutro       0.96      0.94      0.95      4118
    positivo       0.71      0.66      0.68      4119

    accuracy                           0.79     12355
   macro avg       0.79      0.79      0.79     12355
weighted avg       0.79      0.79      0.79     12355



## Teste do modelo SVM

In [179]:
def predict_sentiment(texto):
    texto_teste = tokenizer(texto)
    texto_lematizado = lemmatize(texto_teste)
    text_vect = vect.transform([texto_lematizado])
    return modelo.predict(text_vect)

In [204]:
print(predict_sentiment('Não gostei da ONG, não prestaram o serviço de forma adequada.'))
print(predict_sentiment('Eu adorei ong!'))
print(predict_sentiment('NA OGN poderia oferecer mais serviços, mas ok.'))
print(predict_sentiment('Parece uma boa ONG, contudo poderia ser mais transparente!'))
print(predict_sentiment('Não foi muito bem, ainda precisa melhorar as notas, mas está no caminho'))
print(predict_sentiment('Muito bom!'))
print(predict_sentiment('Muito ruim!'))
print(predict_sentiment('Deveriam ser mais eficientes'))

['negativo']
['positivo']
['positivo']
['negativo']
['positivo']
['positivo']
['negativo']
['positivo']


## Exportando o modelo para uso no Streamlit

In [181]:
arquivo_modelo = "model.pkl"
arquivo_vect = "vect.pkl"

joblib.dump(modelo, arquivo_modelo)
joblib.dump(vect, arquivo_vect)

['vect.pkl']

## Importando o modelo para teste

In [182]:
modelo = joblib.load(arquivo_modelo)
vect = joblib.load(arquivo_vect)

In [207]:
print(predict_sentiment('Parece ok para uma ONG, entretanto, não vi nenhum diferencial, estão na média.'))
print(predict_sentiment('ONG otima'))
print(predict_sentiment('Péssimo'))

['negativo']
['positivo']
['negativo']


# Utilizando NLP para identificar o teor dos comentários dos professores e pedagogos da Passos Mágicos

In [184]:
df = pd.read_csv('/content/processado_base_full.csv', sep=';')
df.head()

,INSTITUICAO_ENSINO_ALUNO,NOME,IDADE_ALUNO,ANOS_PM,FASE_TURMA,PONTO_VIRADA,INDE,INDE_CONCEITO,PEDRA,DESTAQUE_IEG,DESTAQUE_IDA,DESTAQUE_IPV,IAA,IEG,IPS,IDA,IPP,IPV,IAN,ANO,FASE,TURMA,SINALIZADOR_INGRESSANTE,REC_EQUIPE_1,REC_EQUIPE_2,REC_EQUIPE_3,REC_EQUIPE_4,NIVEL_IDEAL,DEFASAGEM,ANO_INGRESSO,BOLSISTA,CG,CF,CT,NOTA_PORT,NOTA_MAT,NOTA_ING,QTD_AVAL,REC_AVA_1,REC_AVA_2,REC_AVA_3,REC_AVA_4,INDICADO_BOLSA
0,Escola Pública,ALUNO-1,11.0,2.0,2H,Não,7.883752,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,8.50002,8.7,7.500,7.000000,5.9375,7.750000,10.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Escola Pública,ALUNO-3,12.0,2.0,3H,Não,7.856390,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,7.91667,8.9,7.500,5.500000,8.1250,8.111113,10.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Escola Pública,ALUNO-4,10.0,2.0,1D,Não,5.076252,D,Quartzo,Ponto a melhorar em 2021: Melhorar a sua entre...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Ponto a melhorar em 2021: Integrar-se mais aos...,8.00002,4.1,6.875,0.000000,7.1875,7.750000,5.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Escola Pública,ALUNO-5,10.0,1.0,2M,Não,8.077085,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Seu destaque em 2020: As suas boas notas na Pa...,Seu destaque em 2020: A sua boa integração aos...,7.50002,8.0,7.500,7.500000,8.4375,8.166665,10.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Rede Decisão/União,ALUNO-8,14.0,3.0,4L,Sim,8.381391,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,8.33334,9.9,4.375,7.333333,8.7500,8.944450,10.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [185]:
df_comentarios_destaque_ieg = df[df['DESTAQUE_IEG'].notna()]['DESTAQUE_IEG'].unique()
df_comentarios_destaque_ieg

array(['Seu destaque em 2020: A sua boa entrega das lições de casa.',
       'Ponto a melhorar em 2021: Melhorar a sua entrega de lições de casa.',
       'Seu destaque em 2020: O seu bom engajamento nas ações de voluntariado.',
       'Ponto a melhorar em 2021: Melhorar o seu engajamento nas ações de voluntariado.',
       'Destaque: A sua boa entrega das lições de casa.',
       'Melhorar: Melhorar a sua entrega de lições de casa.'],
      dtype=object)

In [186]:
df_comentarios_destaque_ida = df[df['DESTAQUE_IDA'].notna()]['DESTAQUE_IDA'].unique()
df_comentarios_destaque_ida

array(['Ponto a melhorar em 2021: Empenhar-se mais nas aulas e avaliações.',
       'Seu destaque em 2020: As suas boas notas na Passos Mágicos.',
       'Seu destaque em 2020: As suas boas notas na Faculdade.',
       'Ponto a melhorar em 2021: As suas notas na Faculdade.',
       'Destaque: As suas boas notas na Passos Mágicos.',
       'Melhorar: Empenhar-se mais nas aulas e avaliações.'], dtype=object)

In [187]:
df_comentarios_destaque_ipv = df[df['DESTAQUE_IPV'].notna()]['DESTAQUE_IPV'].unique()
df_comentarios_destaque_ipv

array(['Seu destaque em 2020: A sua boa integração aos Princípios Passos Mágicos.',
       'Ponto a melhorar em 2021: Integrar-se mais aos Princípios Passos Mágicos.',
       'Destaque: A sua boa integração aos Princípios Passos Mágicos.',
       'Melhorar: Integrar-se mais aos Princípios Passos Mágicos.'],
      dtype=object)

In [188]:
df_comentarios_rec_equipe1 = df[df['REC_EQUIPE_1'].notna()]['REC_EQUIPE_1'].unique()
df_comentarios_rec_equipe1

array(['Promovido de Fase', 'Promovido de Fase + Bolsa', 'Não avaliado',
       'Mantido na Fase atual', 'Mantido na Fase + Bolsa',
       'Alocado em Fase anterior'], dtype=object)

In [189]:
df_comentarios_rec_equipe2 = df[df['REC_EQUIPE_2'].notna()]['REC_EQUIPE_2'].unique()
df_comentarios_rec_equipe2

array(['Promovido de Fase', 'Mantido na Fase atual', 'Não avaliado',
       'Promovido de Fase + Bolsa', 'Mantido na Fase + Bolsa',
       'Alocado em Fase anterior'], dtype=object)

In [190]:
df_comentarios_rec_equipe3 = df[df['REC_EQUIPE_3'].notna()]['REC_EQUIPE_3'].unique()
df_comentarios_rec_equipe3

array(['Mantido na Fase atual', 'Promovido de Fase', 'Não avaliado',
       'Mantido na Fase + Bolsa', 'Promovido de Fase + Bolsa',
       'Alocado em Fase anterior'], dtype=object)

In [191]:
df_comentarios_rec_equipe4 = df[df['REC_EQUIPE_4'].notna()]['REC_EQUIPE_4'].unique()
df_comentarios_rec_equipe4

array(['Não avaliado', 'Mantido na Fase atual', 'Promovido de Fase',
       'Promovido de Fase + Bolsa', 'Mantido na Fase + Bolsa',
       'Alocado em Fase anterior'], dtype=object)

In [192]:
df_comentarios_rec_ava1 = df[df['REC_AVA_1'].notna()]['REC_AVA_1'].unique()
df_comentarios_rec_ava1

array(['Mantido na Fase + Bolsa', 'Promovido de Fase',
       'Mantido na Fase atual', 'Promovido de Fase + Bolsa',
       'Não avaliado', 'Alocado em Fase anterior'], dtype=object)

In [193]:
df_comentarios_rec_ava2 = df[df['REC_AVA_2'].notna()]['REC_AVA_2'].unique()
df_comentarios_rec_ava2

array(['Mantido na Fase atual', 'Promovido de Fase', 'Não avaliado',
       'Promovido de Fase + Bolsa', 'Mantido na Fase + Bolsa'],
      dtype=object)

In [194]:
df_comentarios_rec_ava3 = df[df['REC_AVA_3'].notna()]['REC_AVA_3'].unique()
df_comentarios_rec_ava3

array(['Promovido de Fase', 'Mantido na Fase atual',
       'Promovido de Fase + Bolsa', 'Mantido na Fase + Bolsa',
       'Alocado em Fase anterior'], dtype=object)

In [195]:
df_comentarios_rec_ava4 = df[df['REC_AVA_4'].notna()]['REC_AVA_4'].unique()
df_comentarios_rec_ava4

array(['Promovido de Fase', 'Promovido de Fase + Bolsa',
       'Mantido na Fase atual', 'Mantido na Fase + Bolsa',
       'Alocado em Fase anterior'], dtype=object)

In [196]:
array_nlp_processamento = np.concatenate((df_comentarios_destaque_ieg, df_comentarios_destaque_ida, df_comentarios_destaque_ipv, df_comentarios_rec_equipe1, df_comentarios_rec_equipe2, df_comentarios_rec_equipe3, df_comentarios_rec_equipe4, df_comentarios_rec_ava1, df_comentarios_rec_ava2, df_comentarios_rec_ava3, df_comentarios_rec_ava4))
array_nlp_processamento

array(['Seu destaque em 2020: A sua boa entrega das lições de casa.',
       'Ponto a melhorar em 2021: Melhorar a sua entrega de lições de casa.',
       'Seu destaque em 2020: O seu bom engajamento nas ações de voluntariado.',
       'Ponto a melhorar em 2021: Melhorar o seu engajamento nas ações de voluntariado.',
       'Destaque: A sua boa entrega das lições de casa.',
       'Melhorar: Melhorar a sua entrega de lições de casa.',
       'Ponto a melhorar em 2021: Empenhar-se mais nas aulas e avaliações.',
       'Seu destaque em 2020: As suas boas notas na Passos Mágicos.',
       'Seu destaque em 2020: As suas boas notas na Faculdade.',
       'Ponto a melhorar em 2021: As suas notas na Faculdade.',
       'Destaque: As suas boas notas na Passos Mágicos.',
       'Melhorar: Empenhar-se mais nas aulas e avaliações.',
       'Seu destaque em 2020: A sua boa integração aos Princípios Passos Mágicos.',
       'Ponto a melhorar em 2021: Integrar-se mais aos Princípios Passos Mágicos.'

In [197]:
df_validacao_nlp = pd.DataFrame({
    'texto': array_nlp_processamento
})
df_validacao_nlp['tokenizado'] = df_validacao_nlp['texto'].apply(tokenizer)
df_validacao_nlp['lematizado'] = df_validacao_nlp['tokenizado'].apply(tokenizer)
df_validacao_nlp.head()

,texto,tokenizado,lematizado
0,Seu destaque em 2020: A sua boa entrega das li...,destaque boa entrega licoes casa,destaque boa entrega licoes casa
1,Ponto a melhorar em 2021: Melhorar a sua entre...,ponto melhorar melhorar entrega licoes casa,ponto melhorar melhorar entrega licoes casa
2,Seu destaque em 2020: O seu bom engajamento na...,destaque bom engajamento acoes voluntariado,destaque bom engajamento acoes voluntariado
3,Ponto a melhorar em 2021: Melhorar o seu engaj...,ponto melhorar melhorar engajamento acoes volu...,ponto melhorar melhorar engajamento acoes volu...
4,Destaque: A sua boa entrega das lições de casa.,destaque boa entrega licoes casa,destaque boa entrega licoes casa


In [198]:
df_validacao_nlp['previsao'] = df_validacao_nlp['lematizado'].apply(lambda x: modelo.predict(vect.transform([x])))
df_validacao_nlp.head(50)

,texto,tokenizado,lematizado,previsao
0,Seu destaque em 2020: A sua boa entrega das li...,destaque boa entrega licoes casa,destaque boa entrega licoes casa,[positivo]
1,Ponto a melhorar em 2021: Melhorar a sua entre...,ponto melhorar melhorar entrega licoes casa,ponto melhorar melhorar entrega licoes casa,[positivo]
2,Seu destaque em 2020: O seu bom engajamento na...,destaque bom engajamento acoes voluntariado,destaque bom engajamento acoes voluntariado,[neutro]
3,Ponto a melhorar em 2021: Melhorar o seu engaj...,ponto melhorar melhorar engajamento acoes volu...,ponto melhorar melhorar engajamento acoes volu...,[negativo]
4,Destaque: A sua boa entrega das lições de casa.,destaque boa entrega licoes casa,destaque boa entrega licoes casa,[positivo]
5,Melhorar: Melhorar a sua entrega de lições de ...,melhorar melhorar entrega licoes casa,melhorar melhorar entrega licoes casa,[positivo]
6,Ponto a melhorar em 2021: Empenhar-se mais nas...,ponto melhorar empenhar aulas avaliacoes,ponto melhorar empenhar aulas avaliacoes,[negativo]
7,Seu destaque em 2020: As suas boas notas na Pa...,destaque boas notas passos magicos,destaque boas notas passos magicos,[negativo]
8,Seu destaque em 2020: As suas boas notas na Fa...,destaque boas notas faculdade,destaque boas notas faculdade,[negativo]
9,Ponto a melhorar em 2021: As suas notas na Fac...,ponto melhorar notas faculdade,ponto melhorar notas faculdade,[positivo]


In [199]:
df_validacao_nlp['previsao'].value_counts()

previsao
[positivo]    31
[negativo]    29
[neutro]       1
Name: count, dtype: int64